# Taller práctico: Lógica booleana vs. Árbol de decisión
**Autor:** Alejandro Narváez  
**Fecha de generación automática:** 2025-06-29 17:58

En este cuaderno se compara una regla lógica (dos versiones) con un modelo supervisado (`DecisionTreeClassifier`) para la aprobación de tarjetas de crédito.
Puedes reemplazar la sección de generación de datos sintéticos con tu propio archivo `datos_tarjeta_credito.xlsx` o un CSV en tu Google Drive.

## 1. Instalación e importación de librerías

In [ ]:
!pip install -q scikit-learn pandas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns  # solo para visualización rápida (opcional)
sns.set(style='whitegrid')

## 2. Carga o generación de datos
Si ya tienes un archivo `datos_tarjeta_credito.xlsx` con las columnas `Ingreso_Alto`, `Historial_Credito`, `Empleo_Estable`, `Edad`, `Tarjeta_Aprobada`, comenta o elimina la celda de datos sintéticos y descomenta la celda de carga.

In [ ]:
# --- Opción A: cargar tu propio archivo
# from google.colab import drive
# drive.mount('/content/drive')
# ruta = '/content/drive/MyDrive/datos_tarjeta_credito.xlsx'
# datos = pd.read_excel(ruta)


In [ ]:
# --- Opción B: generar dataset sintético para pruebas ---
np.random.seed(42)
n = 200
datos = pd.DataFrame({
    'Ingreso_Alto': np.random.choice([True, False], n),
    'Historial_Credito': np.random.choice([True, False], n),
    'Empleo_Estable': np.random.choice([True, False], n),
    'Edad': np.random.randint(18, 60, n)
})
# Definimos la etiqueta de forma semi-aleatoria (puedes ajustar la lógica)
datos['Tarjeta_Aprobada'] = (
    (datos['Ingreso_Alto'] & datos['Historial_Credito']) |
    (datos['Empleo_Estable'] & (datos['Edad'] > 30))
).map({True: 'Aprobada', False: 'Rechazada'})

datos.head()

## 3. División de datos y entrenamiento

In [ ]:
X = datos[['Ingreso_Alto', 'Historial_Credito', 'Empleo_Estable', 'Edad']]
y = datos['Tarjeta_Aprobada']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

arbol = DecisionTreeClassifier(random_state=0, max_depth=4)
arbol.fit(X_train, y_train)

pred_test = arbol.predict(X_test)
print('Precisión en test:', accuracy_score(y_test, pred_test))

## 4. Definición de reglas lógicas

In [ ]:
def modelo_decision_v1(ingreso_alto, historial_credito, empleo_estable, edad):
    # Lógica original: (x AND NOT y) OR (y AND NOT z)
    decision = (ingreso_alto and not historial_credito) or (historial_credito and not empleo_estable)
    return 'Aprobada' if decision else 'Rechazada'

def modelo_decision_v2(ingreso_alto, historial_credito, empleo_estable, edad, umbral_edad=25):
    # Versión ampliada con criterio de edad
    decision_base = (ingreso_alto and not historial_credito) or (historial_credito and not empleo_estable)
    decision = decision_base and (edad >= umbral_edad)
    return 'Aprobada' if decision else 'Rechazada'

## 5. Comparación entre el modelo supervisado y las reglas lógicas

In [ ]:
def comparar_modelos(df, preds_supervisado):
    df = df.copy().reset_index(drop=True)
    df['Pred_Supervisado'] = preds_supervisado
    df['Logica_v1'] = df.apply(lambda r: modelo_decision_v1(
        r['Ingreso_Alto'], r['Historial_Credito'], r['Empleo_Estable'], r['Edad']), axis=1)
    df['Logica_v2'] = df.apply(lambda r: modelo_decision_v2(
        r['Ingreso_Alto'], r['Historial_Credito'], r['Empleo_Estable'], r['Edad']), axis=1)
    df['Dif_Superv_vs_v1'] = df['Pred_Supervisado'] != df['Logica_v1']
    df['Dif_v1_vs_v2']     = df['Logica_v1']        != df['Logica_v2']
    return df

resultado = comparar_modelos(X_test, pred_test)

print('Discrepancias Árbol vs Lógica v1:', resultado['Dif_Superv_vs_v1'].sum())
print('Cambios Lógica v1 -> v2:', resultado['Dif_v1_vs_v2'].sum())

# Mostrar ejemplos
resultado.query('Dif_Superv_vs_v1').head()

## 6. Matriz de confusión del árbol

In [ ]:
cm = confusion_matrix(y_test, pred_test, labels=['Aprobada', 'Rechazada'])
fig, ax = plt.subplots(figsize=(4,4))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['Aprobada','Rechazada'], yticklabels=['Aprobada','Rechazada'], ax=ax)
ax.set_xlabel('Predicción'); ax.set_ylabel('Real'); ax.set_title('Matriz de Confusión');
plt.show()

## 7. Conclusiones
- **modelo_decision_v1** es una regla fija que puede no capturar todos los patrones.
- **modelo_decision_v2** restringe con un umbral de edad, modulando el riesgo.
- El **árbol de decisión** aprende relaciones más complejas, aunque podemos afinar hiperparámetros.

Experimenta cambiando el `umbral_edad`, la estructura del árbol (`max_depth`), o integrando otras variables para ver cómo varía la precisión y las discrepancias.